In [1]:
import numpy as np
#import pandas as pd
from keras.callbacks import ModelCheckpoint
#from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras
import keras.backend as K
import tensorflow as tf
from keras import applications
from keras.models import Model
from keras.layers import Flatten, Dense, Input,concatenate,Conv2D,Dropout,GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.models import load_model, model_from_json
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential
from keras.applications.inception_resnet_v2 import InceptionResNetV2
import os
import keras_vggface

Using TensorFlow backend.


In [2]:
pip show keras-vggface

Name: keras-vggface
Version: 0.6
Summary: VGGFace implementation with Keras framework
Home-page: https://github.com/rcmalli/keras-vggface
Author: Refik Can MALLI
Author-email: mallir@itu.edu.tr
License: MIT
Location: c:\users\nsuic_0t98h8a\anaconda3\envs\faceswap\lib\site-packages
Requires: scipy, numpy, keras, six, pyyaml, pillow, h5py
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [18]:
from keras_vggface.vggface import VGGFace
# create a vggface2 model
model = VGGFace(model='resnet50', include_top=True, input_shape=(224, 224, 3), pooling='avg')

In [24]:
for layer in custom_vgg_model.layers[:-3]:
    layer.trainable = False

In [22]:
last_layer = model.get_layer('avg_pool').output
x1 = Flatten(name='flatten')(last_layer)
x2 = Dense(1024, activation='relu')(x1)
x3 = Dense(2, activation='sigmoid', name='classifier')(x2)

custom_vgg_model = Model(model.input, x3)
custom_vgg_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255,)

validation_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)
#this will rescale the images by 1/255

In [8]:
img_h=224
img_w=224

train_generator = train_datagen.flow_from_directory(
#     directory="./Data/SplittedData/Train",
    directory= "dataset/Train",
    target_size=(img_h,img_w),
    color_mode="rgb",
    batch_size=32,
    #class_mode="categorical",
    shuffle=True,
)

validation_generator = test_datagen.flow_from_directory(
    directory="dataset/Validation",
    target_size=(img_h,img_w),
    color_mode="rgb",
    batch_size=16,
   # class_mode="categorical",
    shuffle=False
)


Found 142569 images belonging to 2 classes.
Found 26548 images belonging to 2 classes.


In [25]:
checkpointer = ModelCheckpoint(filepath='Checkpoints\weights.hdf5', monitor='val_loss',verbose=1, save_best_only=True)

history=custom_vgg_model.fit_generator(
                generator=train_generator,
                steps_per_epoch=train_generator.n//train_generator.batch_size,
                 validation_data=validation_generator,
                 validation_steps=validation_generator.n//validation_generator.batch_size,
                epochs=3,
                callbacks=[checkpointer])


Epoch 1/3
4098/4455 [==========================>...] - ETA: 36s - loss: 0.0709 - acc: 0.9744

KeyboardInterrupt: 

In [12]:
custom_vgg_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        input_2[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_50 (Activation)      (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
__________________________________________________________________________________________________
max_poolin

In [26]:
checkpointer2 = ModelCheckpoint(filepath='Checkpoints\weights.hdf5', monitor='val_loss',verbose=1, save_best_only=True)

for layer in custom_vgg_model.layers:
    layer.trainable = True

from keras.optimizers import SGD
custom_vgg_model.compile(optimizer=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True), loss='binary_crossentropy', metrics=['accuracy'])
history2=custom_vgg_model.fit_generator(
                generator=train_generator,
                steps_per_epoch=train_generator.n//train_generator.batch_size,
                 validation_data=validation_generator,
                 validation_steps=validation_generator.n//validation_generator.batch_size,
                epochs=15,
                callbacks=[checkpointer2]  )

Epoch 1/15
4455/4455 [==============================] - 1752s 393ms/step - loss: 0.4204 - acc: 0.8562 - val_loss: 0.3786 - val_acc: 0.8735

Epoch 00001: val_loss improved from inf to 0.37860, saving model to Checkpoints\weights.hdf5
Epoch 2/15
4455/4455 [==============================] - 1744s 392ms/step - loss: 0.4097 - acc: 0.8571 - val_loss: 0.4051 - val_acc: 0.8735

Epoch 00002: val_loss did not improve from 0.37860
Epoch 3/15
4455/4455 [==============================] - 1734s 389ms/step - loss: 0.4090 - acc: 0.8572 - val_loss: 0.3763 - val_acc: 0.8735

Epoch 00003: val_loss improved from 0.37860 to 0.37631, saving model to Checkpoints\weights.hdf5
Epoch 4/15
4455/4455 [==============================] - 1734s 389ms/step - loss: 0.4106 - acc: 0.8565 - val_loss: 0.3796 - val_acc: 0.8735

Epoch 00004: val_loss did not improve from 0.37631
Epoch 5/15
4455/4455 [==============================] - 1735s 389ms/step - loss: 0.4097 - acc: 0.8564 - val_loss: 0.3787 - val_acc: 0.8735

Epoch 00